# 파이단틱 사용하기 

파이단틱은 입력된 데이터의 유효성과 형식을 검증하고 특정 데이터 형식으로 명확하게 표현할 때 사용하는 라이브러리 



In [1]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    # Base 모델은 ticker가 문자열인지, period가 문자열인지, 필수 값이 비었는지 검증
    # ...은 반드시 있어야 하는 값 
    # description은 필드에 대한 설명
    # title은 필드의 제목
    ticker : str = Field(..., title = '주식 코드', description="주식 코드 ( 예 : AAPL )")
    period: str  = Field(..., title = '기간', description="주식 데이터 조회 기간( 예 : 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max )")

🪄 실제 사용 예시
data = StockHistoryInput(ticker="AAPL", period="1y")
print(data)


그러면 이런 객체가 생성됨:

ticker='AAPL' period='1y'


반대로 이런 입력이면:

StockHistoryInput(ticker=123, period=1)


➡️ 자동으로 에러 발생
(타입이 str이 아니라서)

In [2]:
!pip install yfinance


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import yfinance as yf
import pytz
from datetime import datetime
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
import dotenv
import json


dotenv.load_dotenv()


@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """
    주식 종목의 각각 데이터를 조회하는 함수 

    """
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown()
    return history_md


@tool
def get_current_time(timezone: str, location: str) -> str:
    """현재 시각을 반환하는 함수
    Args:
        timezone (str): 시간대 ( 예: 'Asia/Seoul' ). 실제 존재하는 시간대여야함.
        location (str): 지역명. 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_time = f"{timezone} , {location}의 현재 시각은 {now} 입니다."
    print(location_and_time)
    return location_and_time

llm = ChatOpenAI(model="gpt-4o")

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}  

llm_with_tools = llm.bind_tools(tools)

In [8]:
# 회사의 주가 변화에 관한 질문을 추가하고, llm_with_tools.invoke를 사용하여 답변을 요청
messages = []
messages.append(HumanMessage(content="테슬라는 한달 전에 비해 주가가 올랐어? 내렸어?"))

# chain 설정
parser = StrOutputParser()
chain = llm_with_tools | parser
chain.invoke(messages)

# 체인을 설정하지 않고, ai의 response 출력 결과 보기
response = llm_with_tools.invoke(messages)
print(response.tool_calls)
# [{'name': 'get_yf_stock_history', 
# 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 
# 'id': 'call_lAs1CtuqjTeePDIHt4FtM2t6', 'type': 'tool_call'}]

messages.append(response)

# messages 안에는 이미 아래와 같이 들어가 있으므로, ai가 어떠한 주체가 답변을 했는지 명확히 알 수 있다.
# {
#   "type": "ai",
#   "content": "안녕하세요! 무엇을 도와드릴까요?"
# }

[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_UKQYvK5oaqHgiVoMa7xmErl4', 'type': 'tool_call'}]


In [9]:
# gpt가 의도한 대로 함수를 실행하고 결과를 메시지로 추가할 수 있도록 다음과 같이 코드를 만듬
# 위에 출력된 결과를 보면 reponse안의 tool_calls에 리스트 형태로 실행할 함수들이 들어 있는 형태였다.
# 따라서 reponse.tool_calls로 for문을 이용해 gpt가 계획했던 함수를 모두 실행하고 그 결과들을 messages에 추가


for tool_call in response.tool_calls:
    # tool_dict[tool_call["name"]] 부분은 tool 객체인 
    # name='get_yf_stock_history' 
    # description='주식 종목의 각각각 데이터를 조회하는 함수' 
    # args_schema=<class 'langchain_core.utils.pydantic.get_yf_stock_history'> 
    # func=<function get_yf_stock_history at 0x...>

    selected_tool = tool_dict[tool_call["name"]]
    print(selected_tool)
    # print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    # print(tool_msg)
llm_with_tools.invoke(messages)

name='get_yf_stock_history' description='주식 종목의 각각 데이터를 조회하는 함수' args_schema=<class 'langchain_core.utils.pydantic.get_yf_stock_history'> func=<function get_yf_stock_history at 0x000001E16561EC00>


AIMessage(content='테슬라 주가는 한 달 전인 2025년 10월 22일에 438.97 달러로 마쳤는데, 가장 최근인 2025년 11월 21일에 391.09 달러로 마감되었습니다. 따라서, 테슬라의 주가는 한 달 전보다 내렸습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1636, 'total_tokens': 1709, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_c98e05ca17', 'id': 'chatcmpl-CecDTv11XglAn2Jvn7eFiXKxp8BRS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--1d1b8a77-9109-43cf-8154-b73d4de3a00c-0', usage_metadata={'input_tokens': 1636, 'output_tokens': 73, 'total_tokens': 1709, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
# 데코레이터

# 1초 뒤, 문자열 출력
import time 
def timer(func):
    def wrapper(*args):
        time.sleep(3)
        return func(*args)
    return wrapper
@timer
def print_text(text):
    print(text)

print_text("Hello, World!")

Hello, World!


## @tool 데코레이터가 하는 일

원래는 이 함수가 그냥 파이썬 함수였습니다.

def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    ...


여기에 @tool을 붙이면:

@tool
def get_yf_stock_history(...):
    ...


LangChain이 이 함수를 감싸서, 내부적으로 다음과 같은 정보를 가진 Tool 객체로 바꿔 줍니다.

name : "get_yf_stock_history"

description : docstring

args_schema : Pydantic 기반 입력 스키마

func : 실제 파이썬 함수

.invoke(...) : 이 도구를 호출하는 메서드

In [11]:
# 스트림 모드로 출력하기 

for c in llm.stream([HumanMessage('잘 지냈어? 한국 사회의 문제점이 무엇인지 이야기해줘')]):
    print(c.content , end='|')

|안|녕하세요|!| 네|,| 잘| 지|내|고| 있습니다|.| 한국| 사회|는| 많은| 발전|을| 이루|었|지만|,| 여|전히| 해결|해야| 할| 여러| 가지| 문제|점|이| 존재|합니다|.| 몇| 가지| 주요| 문제|점을| 살|펴|보|면| 다음|과| 같습니다|.

|1|.| **|고|령|화| 문제|**|:| 한국|은| 세계|에서| 가장| 빠|른| 속|도로| 고|령|화|가| 진행|되고| 있습니다|.| 이는| 노동|력| 감소|,| 연|금| 제|도| 부담| 증가|,| 그리고| 세|대| 간| 갈|등| 등의| 문제|를| 초|래|하고| 있습니다|.

|2|.| **|저|출|산| 문제|**|:| 출|산|율|이| 매우| 낮|아| 인|구| 감소|와| 향|후| 경제| 성장|에| 부|정|적인| 영향을| 미|칠| 가능|성이| 큽|니다|.| 젊|은| 세|대|가| 자|녀|를| 갖|기| 어려|워|하는| 경제|적|,| 사회|적| 요|인|들이| 문제|의| 원|인|으로| 지|목|되고| 있습니다|.

|3|.| **|교육| 경쟁|과| 입|시| 스트|레스|**|:| 과|도|한| 교육|열|과| 경쟁|으로| 인해| 학생|들이| 큰| 스트|레|스를| 받고| 있으며|,| 이|로| 인해| 정신| 건강| 문제|도| 발생|하고| 있습니다|.

|4|.| **|주|택| 문제|**|:| 집|값| 상승|으로| 인해| 많은| 사람들이| 주|거|비| 부담|을| 느|끼|고| 있으며|,| 특히| 젊|은|층|과| 신|혼|부|부|들이| 내| 집| 마련|에| 어려|움을| 겪|고| 있습니다|.

|5|.| **|환경| 문제|**|:| 미|세|먼|지|와| 같은| 대|기|오|염| 문제가| 건강|에| 악|영|향|을| 미|치|고| 있으며|,| 지속| 가능한| 환경| 정책|이| 필요|합니다|.

|6|.| **|정|치|적| 양|극|화|**|:| 정치|적| 양|극|화|와| 이|로| 인|한| 사회|적| 갈|등|이| 점|차| 심|화|되고| 있으며|,| 이는| 사회| 통|합|에| 큰| 도|전|이| 되고| 있습니다|.

|이| 외|에도| 다양한

In [17]:
messages= [ 
    SystemMessage('너는 사용자의 질문에 답변을 하기 위해 toools를 사용할 수 있다.'),
    HumanMessage('부산은 지금 몇시야?'),
]
response = llm_with_tools.stream(messages)

is_first = True
for chunck in response:
    print('chunck type:', type(chunck))
    if is_first:
        is_first = False # 첫실행인지 확인하는 플래그
        gathered = chunck # 첫 청크를 gathered에 할당
    else:
        gathered += chunck
    print('content: ',gathered.content, 'tool_call_chunk', gathered.tool_calls, end='|')
messages.append(gathered)

chunck type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_ihqEY2YOnzzA50CGnm2nE2wA', 'type': 'tool_call'}]|chunck type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_ihqEY2YOnzzA50CGnm2nE2wA', 'type': 'tool_call'}]|chunck type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_ihqEY2YOnzzA50CGnm2nE2wA', 'type': 'tool_call'}]|chunck type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_ihqEY2YOnzzA50CGnm2nE2wA', 'type': 'tool_call'}]|chunck type: <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_ihqEY2YOnzzA50CGnm2nE2wA', 'type': 'tool_c

In [ ]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과 반환 
    messages.append(tool_msg)


{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul , 부산의 현재 시각은 2025-11-22 16:47:44 입니다.


# 스트림릿에 구현 

In [ ]:
import streamlit as st

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.chat_historys import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


st.title('chat bot')

messages = [
    SystemMessage(content="너는 친절한 챗봇이야."),
]

if 'messages' not in st.session_state:
    st.session_state['messages'] = messages

if 'store' not in st.session_state:
    st.session_state['store'] = {}

def get_session_history(session_id:str):
    if session_id not in st.session_state['store']:
        st.session_state['store'][session_id] = InMemoryChatMessageHistory()
    return st.session_state["store"][session_id]  # InMemoryChatMessageHistory 객체 반환

for msg in st.session_state['messages']:
    if msg:
        if isinstance(msg, SystemMessage):
            st.chat_message("system").write(msg.content)
        elif isinstance(msg, HumanMessage):
            st.chat_message("user").write(msg.content)
        elif isinstance(msg, AIMessage):
            st.chat_message("assistant").write(msg.content)

In [ ]:
coinfig = {'configurable' : {'session_id' : 'abc123'}}
